To do:
- beam center finder flatten issue
- zoom files via scippneutron.load_with_mantid
- load masks with mantid?
- `direct_beam = loki.io.load_rkh_wav(f'{path}/DirectBeam_20feb_full_v3.dat')`
- make providers from mantid loaders
- add files to pooch for testing?
```
mask_file = f'{path}/mask_new_July2022.xml'
mask_all = scn.load(filename=idf_filename, mantid_alg="LoadMask", mantid_args={"InputFile": mask_file})
```

In [ ]:
from mantid.simpleapi import *

root = '/home/simon/instruments/zoom/ZOOM_Scipp'
name86 = 'ZOOM00034786.nxs'

ws, mons = Load(Filename=f'{root}/{name86}', LoadMonitors=True)

# import scippneutron as scn
# scn.from_mantid(mons)['data']
ws

In [ ]:
from mantid.simpleapi import Load, LoadInstrument
import scippneutron as scn

root = '/home/simon/instruments/zoom/ZOOM_Scipp'


def to_scipp_hdf5(name: str):
    # ws, mon = Load(Filename=f'{root}/{name}', LoadMonitors=True)
    # cal = Load(Filename=f'{root}/192tubeCalibration_11-02-2019_r5_10lines.nxs')
    # dg = scn.from_mantid(ws)
    dg = scn.load_with_mantid(f'{root}/{name}', mantid_args={'LoadMonitors': True})
    dg.save_hdf5(f'{root}/{name}.hdf5')


to_scipp_hdf5('ZOOM00034786.nxs')
to_scipp_hdf5('ZOOM00034787.nxs')
to_scipp_hdf5('192tubeCalibration_11-02-2019_r5_10lines.nxs')

In [ ]:
import scippnexus as snx

with snx.File(f'{root}/{name86}', 'r') as f:
    dg86 = f['raw_data_1'][()]

dg86

# Sans2d data reduction

## Introduction

This notebook gives a concise overview of how to use the `esssans` package with Sciline, on the example of the data reduction of a Sans2d experiment.
We begin with relevant imports:

In [ ]:
import numpy as np
import scipp as sc
import sciline
import scippneutron as scn
import plopp as pp
import esssans as sans
from esssans.types import *

In [ ]:
root = '/home/simon/instruments/zoom/ZOOM_Scipp'

run86 = sc.io.load_hdf5(f'{root}/ZOOM00034786.nxs.hdf5')
run87 = sc.io.load_hdf5(f'{root}/ZOOM00034787.nxs.hdf5')
cal = sc.io.load_hdf5(f'{root}/192tubeCalibration_11-02-2019_r5_10lines.nxs.hdf5')

run86['data'].coords["gravity"] = sans.common.gravity_vector()

In [ ]:
def load_direct_beam(filename: DirectBeamFilename) -> DirectBeam:
    db = scn.load(
        filename=filename,
        mantid_alg='LoadRKH',
        mantid_args={'FirstColumnValue': 'Wavelength'},
    )
    return DirectBeam(db)

In [ ]:
params = sans.sans2d.default_parameters.copy()
# TODO Check monitor names
params[NeXusMonitorName[Incident]] = 'monitor3'
params[NeXusMonitorName[Transmission]] = 'monitor5'

# params[FileList[BackgroundRun]] = ['SANS2D00063159.hdf5']
# params[FileList[SampleRun]] = ['SANS2D00063114.hdf5']
# params[FileList[EmptyBeamRun]] = ['SANS2D00063091.hdf5']
params[LoadedFileContents[SampleRun]] = run86
# params[LoadedFileContents[BackgroundRun]]
params[LoadedFileContents[EmptyBeamRun]] = run87
# params[LoadedFileContents[TransmissionRun[BackgroundRun]]] = params[
#    LoadedFileContents[BackgroundRun]
# ]
params[LoadedFileContents[TransmissionRun[SampleRun]]] = params[
    LoadedFileContents[SampleRun]
]
params[DirectBeamFilename] = 'Direct_Zoom_4m_8mm_100522.txt'

# TODO How to load calibration
# TODO How to load or apply masks?
# TODO What is the moderator file?

params[OutFilename] = 'reduced.nxs'

band = sc.linspace('wavelength', 1.75, 16.5, num=2, unit='angstrom')
params[WavelengthBands] = band
params[WavelengthBins] = sc.linspace(
    'wavelength', start=band[0], stop=band[-1], num=141
)

params[sans.sans2d.LowCountThreshold] = sc.scalar(100, unit='counts')

mask_interval = sc.array(dims=['wavelength'], values=[2.21, 2.59], unit='angstrom')
params[WavelengthMask] = sc.DataArray(
    sc.array(dims=['wavelength'], values=[True]),
    coords={'wavelength': mask_interval},
)

params[QBins] = sc.linspace(dim='Q', start=0.004, stop=0.8, num=141, unit='1/angstrom')
params[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
params[CorrectForGravity] = False
params[UncertaintyBroadcastMode] = UncertaintyBroadcastMode.upper_bound

In [ ]:
providers = sans.providers + sans.sans2d.providers
pipeline = sciline.Pipeline(providers, params=params)
pipeline.insert(load_direct_beam)

In [ ]:
# da = pipeline.get(RawData[SampleRun]).compute()
da = pipeline.get(CalibratedMaskedData[SampleRun]).compute()
scn.instrument_view(da.hist(), pixel_size=0.01, norm='log')

In [ ]:
da

In [ ]:
da = pipeline.get(RawData[BackgroundRun]).compute()
scn.instrument_view(da.hist(), pixel_size=0.01, norm='log')

In [ ]:
# iofq = pipeline.get(BackgroundSubtractedIofQ)
iofq = pipeline.get(IofQ[SampleRun])

In [ ]:
%matplotlib widget
result = iofq.compute()
result.plot()

In [ ]:
iofq.visualize(graph_attr={'rankdir': 'LR'})

### Compute intermediate results

For inspection and debugging purposes we can also compute intermediate results.
To avoid repeated computation (including costly loading of files) we can request multiple results at once, including the final result, if desired.
For example:

In [ ]:
monitors = (
    WavelengthMonitor[SampleRun, Incident],
    WavelengthMonitor[SampleRun, Transmission],
    # WavelengthMonitor[BackgroundRun, Incident],
    # WavelengthMonitor[BackgroundRun, Transmission],
)
parts = (CleanSummedQ[SampleRun, Numerator], CleanSummedQ[SampleRun, Denominator])
iofqs = (IofQ[SampleRun],)
keys = monitors + (MaskedData[SampleRun],) + parts + iofqs

results = pipeline.compute(keys)

display(sc.plot({str(key): results[key] for key in monitors}, norm='log'))

display(
    scn.instrument_view(
        results[MaskedData[SampleRun]].hist(),
        pixel_size=0.0075,
        norm='log',
        camera=pp.graphics.Camera(position=(0, 0, 22)),
    )
)

parts = {str(key): results[key] for key in parts}
parts = {key: val if val.bins is None else val.hist() for key, val in parts.items()}
display(sc.plot(parts, norm='log'))

iofqs = {str(key): results[key] for key in iofqs}
iofqs = {key: val if val.bins is None else val.hist() for key, val in iofqs.items()}
display(sc.plot(iofqs, norm='log'))